In [1]:
from __future__ import print_function, division, unicode_literals
import numpy as np
import codecs
import random
import nltk
from time import time

In [2]:
%%time

from spacy.en import English
nlp = English()

CPU times: user 1min 8s, sys: 1.08 s, total: 1min 9s
Wall time: 1min 10s


In [13]:
for w in nlp('In order for boulders to become sediments, the boulders must first be weathered and eroded.'):
    print(w.text, '->', w.head, [c.text for c in w.children], w.dep_, w.tag_, [s for s in w.subtree])

In -> weathered  [u'order'] prep IN [In , order , for , boulders , to , become , sediments]
order -> In  [u'become'] pobj NN [order , for , boulders , to , become , sediments]
for -> become  [] mark IN [for ]
boulders -> become  [] nsubj NNS [boulders ]
to -> become  [] aux TO [to ]
become -> order  [u'for', u'boulders', u'to', u'sediments'] advcl VB [for , boulders , to , become , sediments]
sediments -> become  [] attr NNS [sediments]
, -> weathered  [] punct , [, ]
the -> boulders  [] det DT [the ]
boulders -> weathered  [u'the'] nsubjpass NNS [the , boulders ]
must -> weathered  [] aux MD [must ]
first -> weathered  [] advmod RB [first ]
be -> weathered  [] auxpass VB [be ]
weathered -> weathered  [u'In', u',', u'boulders', u'must', u'first', u'be', u'and', u'eroded', u'.'] ROOT VBN [In , order , for , boulders , to , become , sediments, , , the , boulders , must , first , be , weathered , and , eroded, .]
and -> weathered  [] cc CC [and ]
eroded -> weathered  [] conj VBN [eroded]


In [3]:
# DATA = "/Users/Pavel/Code/allen-ai-challenge/data/ck12_tokens.txt"
DATA = "/home/marat/ck12_tokens.txt"
TRAINING_SET_MERGED = "/home/marat/Downloads/training_set_merged.tsv"

In [4]:
GLOVE_D = 300

In [5]:
GLOVE_FILE = "/home/marat/data/glove.6B/glove.6B.%dd.txt" % GLOVE_D
GLOVE_FILE = "/home/marat/data/word2vec.txt"

In [ ]:
def read_dict(fname):
    d = {}
    with codecs.open(fname, encoding="utf-8") as f:
        for row in (line.strip().split() for line in f):
            w = row[0]
            v = np.array(row[1:], dtype='float32')
            assert v.shape == (GLOVE_D,)
            d[w] = v
    return d

In [ ]:
GLOVE = read_dict(GLOVE_FILE)

In [ ]:
len(GLOVE)

In [6]:
# def text2vec(text, glove_dict, seq_length, debug=False):
#     vecs = []
#     for w in text.split():
#         try:
#             vecs.append(glove_dict[w][np.newaxis, :])
#         except KeyError:
#             if debug:
#                 print("%s is not found" % w)
#             continue
#     if not vecs:
#         print("no vector for '%s'" % text)
#         return np.zeros((seq_length, GLOVE_D))
    
#     rec = np.concatenate(vecs, axis=0).astype('float32')
#     if rec.shape[0] > seq_length:
#         # trim long sentences
#         rec = rec[rec.shape[0] - seq_length:, :]
#     elif rec.shape[0] < seq_length:
#         # extend short sentences with zeros
#         rec = np.vstack([np.zeros((seq_length - rec.shape[0], rec.shape[1])), rec])
#     assert rec.shape[0] == seq_length
#     return rec

def text2vec(text, seq_length, debug=False):
    D = 300
    vecs = []
    for w in text:
        vecs.append(w.vector[np.newaxis])

    if not vecs:
        print("no vector for '%s'" % text)
        return np.zeros((seq_length, D), dtype='float32')
    
    rec = np.concatenate(vecs, axis=0)
    if rec.shape[0] > seq_length:
        # trim long sentences
        rec = rec[rec.shape[0] - seq_length:, :]
    elif rec.shape[0] < seq_length:
        # extend short sentences with zeros
        rec = np.vstack([np.zeros((seq_length - rec.shape[0], rec.shape[1])), rec])
    assert rec.shape[0] == seq_length
    return rec.astype('float32')

In [7]:
# text2vec(next(nlp("hello , die").sents), 3)

In [8]:
# %%time
# i=0
# sentences = []
# with codecs.open(DATA, encoding="utf-8") as f:
#     for line in f:
#         if u'↓' in line:
#             continue
# #         i += 1
# #         if 63798 < i 63800 :
#         sentences.append(line.strip())
        
# blob = TextBlob(' '.join(sentences))
# all_pos_tags = blob.tags        

# i = 0
# pos_tags = []
# for l, line in enumerate(sentences):    
#     tags = []
#     for word in line.split():
# #         if l == 63798:
# #         print(word, i, all_pos_tags[i])
#         if word.replace(".", "") == all_pos_tags[i][0].replace(".", ""):
#             tags.append(all_pos_tags[i][1])
#             i += 1
#         else:
#             tags.append('.')
            
#     pos_tags.append(tags)
# #     if l == 63798:
# #     print(line, tags)
#     break

# del all_pos_tags

# assert len(sentences) == len(pos_tags)

In [9]:
%%time
sentences = []
with codecs.open(DATA, encoding="utf-8") as f:
    for i, line in enumerate(f):
        if u'↓' in line:
            continue
        for s in nlp(line.strip()).sents:
            sentences.append(s)

CPU times: user 1min 42s, sys: 416 ms, total: 1min 42s
Wall time: 1min 42s


In [10]:
# from spacy import parts_of_speech as pos
# from spacy.util import DEP
# # print(sentences[0])
# for i, s in enumerate(sentences[10000:]):
#     if i >= 100:
#         break
# #     if i != 7:
# #         continue
#     has_conj = False
#     for w in s:
#         if 'conj' in w.dep_:
#             has_conj = True
#             break
#     if not has_conj:
#         continue
    

#     print(s.text)
# #     s = next(nlp(s).sents)

#     subj = None
#     for w in s:
#         if 'nsubj' in w.dep_:
#             subj = w
# #         print('\t', w.text, w.dep_, w.tag_)

#     for w in s:
#         if 'conj' in w.dep_:
#             print(w, w.tag_, '&', w.head, w.head.tag_, '->', w.head.head, w.head.head.tag_)

#     for n in s.doc.noun_chunks:
#         print('*', n)
        

#     print(subj, s.root)
#     print('-'*80)

CC Coordinating conjunction
CD Cardinal number
DT Determiner
EX Existential there
FW Foreign word
IN Preposition or subordinating conjunction
JJ Adjective
JJR Adjective, comparative
JJS Adjective, superlative
LS List item marker
MD Modal
NN Noun, singular or mass
NNS Noun, plural
NNP Proper noun, singular
NNPS Proper noun, plural
PDT Predeterminer
POS Possessive ending
PRP Personal pronoun
PRP$ Possessive pronoun
RB Adverb
RBR Adverb, comparative
RBS Adverb, superlative
RP Particle
SYM Symbol
TO to
UH Interjection
VB Verb, base form
VBD Verb, past tense
VBG Verb, gerund or present participle
VBN Verb, past participle
VBP Verb, non­3rd person singular present
VBZ Verb, 3rd person singular present
WDT Wh­determiner
WP Wh­pronoun
WP$ Possessive wh­pronoun
WRB Wh­adverb

In [11]:
CORRUPT_WINDOW = 10
# def corrupt(sentences, index):
#     s = sentences[index]
#     noun_indices = [i for i, w in enumerate(s) if 'NN' in w.tag_]
#     if not noun_indices:
#         return None
#     noun_to_replace_index = random.choice(noun_indices)
#     all_donor_indices = range(max(0, index-CORRUPT_WINDOW), index) + \
#                         range(index+1, min(len(sentences), index+CORRUPT_WINDOW+1))
#     donor_index = random.choice(all_donor_indices)
#     all_nouns_to_insert = [w for w in sentences[donor_index] if 'NN' in w.tag_]
#     if not all_nouns_to_insert:
#         return None
#     noun_to_insert = random.choice(all_nouns_to_insert)
    
#     return next(nlp(s[:noun_to_replace_index].text + ' ' + noun_to_insert.text + ' ' 
#                     + s[noun_to_replace_index+1:].text).sents)

def corrupt(sentences, index):
    s = sentences[index]
    noun_chunks = list(s.doc.noun_chunks)
    if not noun_chunks:
        return None
    noun_chunk_to_replace = random.choice(noun_chunks)
    
    all_donor_indices = range(max(0, index-CORRUPT_WINDOW), index) + \
                        range(index+1, min(len(sentences), index+CORRUPT_WINDOW+1))
    donor_index = random.choice(all_donor_indices)
    all_noun_chunks_to_insert = list(sentences[donor_index].doc.noun_chunks)
    if not all_noun_chunks_to_insert:
        return None
    np_to_insert = random.choice(all_noun_chunks_to_insert)
    
    new_text = s[:noun_chunk_to_replace.start].text + ' ' + np_to_insert.text + ' ' \
                    + s[noun_chunk_to_replace.end:].text
    
    return next(nlp(new_text).sents)

In [12]:
# %%time
# for c in range(len(sentences)):
#     corrupted = corrupt(sentences, c)
#     if corrupted:
#         text2vec(sentences[c], SEQ_LENGTH), text2vec(corrupted, SEQ_LENGTH)

In [16]:
%%time
import theano
import theano.tensor as T
import lasagne
from lasagne.nonlinearities import very_leaky_rectify, leaky_rectify
theano.config.floatX = 'float32'

SEQ_LENGTH = 30
MARGIN = 1

l_in = lasagne.layers.InputLayer(shape=(None, GLOVE_D, SEQ_LENGTH))
n = lasagne.layers.GRULayer()
# n = lasagne.layers.Conv1DLayer(l_in, 30, filter_size=3, nonlinearity=very_leaky_rectify)  # None x 10 x 28
# n = lasagne.layers.MaxPool1DLayer(n, 2)  # None x f x 14
# n = lasagne.layers.Conv1DLayer(l_in, 40, filter_size=3, nonlinearity=very_leaky_rectify)  # None x 20 x 12
# n = lasagne.layers.MaxPool1DLayer(n, 2)  # None x f x 6
# n = lasagne.layers.Conv1DLayer(l_in, 50, filter_size=3, nonlinearity=leaky_rectify)  # None x 30 x 4
# n = lasagne.layers.MaxPool1DLayer(n, 2)  # None x f x 2
n = lasagne.layers.reshape(n, ([0], -1))
# n = lasagne.layers.DropoutLayer(n, 0.5)
n = lasagne.layers.DenseLayer(n, 100)
n = lasagne.layers.DenseLayer(n, 1)

output = lasagne.layers.get_output(n)
params = lasagne.layers.get_all_params(n)

correct_energy = output[0::2][0]  # 50
corrupt_energy = output[1::2][0]  # 50

# energy = T.maximum(0, MARGIN + correct_energy - corrupt_energy).mean()  # hinge loss
# energy = (correct_energy**2).mean() + ((T.maximum(0, MARGIN - corrupt_energy))**2).mean()  # square-square loss
energy = (correct_energy**2).mean() + (MARGIN * T.exp(-corrupt_energy)).mean()  # square-exponential
updates = lasagne.updates.adam(energy, params)

print('Compiling functions ...')
forward_fn = theano.function([l_in.input_var], output)
train_fn = theano.function([l_in.input_var], energy, updates=updates)
test_fn = theano.function([l_in.input_var], energy)

Compiling functions ...
CPU times: user 2.47 s, sys: 260 ms, total: 2.73 s
Wall time: 10.7 s


In [17]:
def energy_fn(txt):
    sents = nlp(txt).sents
    data = [text2vec(s, SEQ_LENGTH).T[np.newaxis] for s in sents]
    return forward_fn(np.concatenate(data, axis=0)).mean()

In [18]:
%%time
BATCH_SIZE = 50
EPOCH_COUNT = 100
indices = np.arange(len(sentences))
for e in range(EPOCH_COUNT):
    epoch_start = time()
    
    tries = []
    with codecs.open(TRAINING_SET_MERGED, encoding="utf-8") as f:
        for i, l in enumerate(f):
            q_id, correct, a1, a2, a3, a4 = l.strip().split("\t")
            energies = [energy_fn(v) for v in [a1, a2, a3, a4]]
            guess = "ABCD"[np.argmin(energies)]
            tries.append(guess == correct)   
    
    np.random.shuffle(indices)
    errors = []
    for i in xrange(0, indices.shape[0], BATCH_SIZE):
        train_sent_idx = [k for k in indices[i:i+BATCH_SIZE]]
        train_data = []
        for correct_idx in train_sent_idx:
            corrupted = corrupt(sentences, correct_idx)
            if corrupted:
                train_data.append(text2vec(sentences[correct_idx], SEQ_LENGTH).T[np.newaxis])
                train_data.append(text2vec(corrupted, SEQ_LENGTH).T[np.newaxis])
        train_data = np.concatenate(train_data, axis=0)
        error = train_fn(train_data)
        errors.append(error)
                   
    time_passed = time() - epoch_start
    print(e, np.mean(errors), '%.1f%%' % (np.mean(tries) * 100),'%.0fs' % time_passed)

0 0.832509 24.3% 326s
1 0.828645 25.1% 356s
2 0.827393 24.1% 322s
3 0.826316 25.6% 330s
4 0.824849 26.1% 325s
5 0.825799 25.0% 325s
6 0.82316 24.4% 322s
7 0.823376 24.0% 322s
8 0.820097 23.4% 322s
9 0.825687 24.7% 324s
10 0.818421 23.7% 323s
11 0.822294 25.0% 322s
12 0.824571 23.9% 321s
13 0.825725 24.8% 327s
14 0.820987 25.8% 327s
15 0.820714 24.9% 331s
16 0.819233 24.7% 321s
17 0.823632 23.9% 321s
18 0.822845 24.1% 323s
19 0.819962 25.6% 325s
20 0.817265 25.0% 325s
21 0.816961 24.3% 324s
22 0.816117 23.7% 320s
23 0.837103 24.9% 325s
24 0.823386 24.4% 324s
25 0.81867 24.9% 326s
26 0.817251 25.6% 322s
27 0.817315 25.7% 323s
28 0.818032 23.2% 324s
29 0.8147 25.2% 325s
30 0.819606 24.1% 325s
31 0.815127 23.5% 325s
32 0.806638 24.8% 325s
33 0.815123 24.0% 322s
34 0.813973 23.0% 320s
35 0.814883 22.4% 320s
36 0.811355 24.2% 321s
37 0.809877 23.8% 321s
38 0.808651 23.8% 323s
39 0.806798 24.2% 321s
40 0.804566 23.8% 322s
41 0.816157 23.8% 321s
42 0.803599 23.3% 322s
43 0.804702 23.6% 323s
44

In [ ]:
type(sentences[0])

In [ ]:
v1 = 'the sun is the main source of energy for the water cycle .'
v2 = 'fossil fuels is the main source of energy for the water cycle .'
v3 = 'clouds is the main source of energy for the water cycle .'
v4 = 'the ocean is the main source of energy for the water cycle .'

# v1 = 'tension has the greatest effect on aiding the movement of blood through the human body .'
# v2 = 'friction has the greatest effect on aiding the movement of blood through the human body .'
# v3 = 'density has the greatest effect on aiding the movement of blood through the human body .'
# v4 = 'gravity has the greatest effect on aiding the movement of blood through the human body .'

print(energy_fn(v1))
print(energy_fn(v2))
print(energy_fn(v3))
print(energy_fn(v4))

In [ ]:
%%time

tries = []

with codecs.open(TRAINING_SET_MERGED, encoding="utf-8") as f:
    for i, l in enumerate(f):
        q_id, correct, a1, a2, a3, a4 = l.strip().split("\t")
        energies = [energy_fn(v) for v in [a1, a2, a3, a4]]
        guess = "ABCD"[np.argmin(energies)]
#         print(guess, correct, q_id, zip([a1, a2, a3, a4], energies))
        tries.append(guess == correct)
#         if i > 10:
#             break

In [ ]:
np.mean(tries)